In [5]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [6]:
#imports
import numpy as np
import pandas as pd
import os
import re
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras import backend as KerasTensor

import keras_tuner as kt


In [7]:
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Load Dataset
data = pd.read_csv("train.csv")

In [9]:
data.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [12]:
tweet = list(data['tweet'])

In [15]:
tweet

["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
 '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
 '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
 '!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny',
 '!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
 '!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"',
 '!!!!!!"@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"',
 "!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;",
 '" &amp; you might not get ya bitch back &amp; thats t

In [16]:
labels = list(data['class'])

In [18]:
#Pre-Processing
stop_words = set(stopwords.words('english'))
stop_words.add("rt")

import nltk
nltk.download('punkt_tab') # Download the missing resource

def remove_entity(raw_text):
  return re.sub(r"&[^\s;]+;","",raw_text)

def change_user(raw_text):
  return re.sub(r"@([^ ]+)","user",raw_text)

def remove_url(raw_text):
  url_regex = r"http\S+|www\S+|https\S+"
  return re.sub(url_regex,'',raw_text)

def remove_noise_symbols(raw_text):
  return raw_text.replace('"','').replace("'",'').replace('!','').replace('`','').replace('..','')

def remove_stopwords(raw_text):
  tokens = nltk.word_tokenize(raw_text)
  filtered = [w for w in tokens if w.lower() not in stop_words]
  return " ".join(filtered)

def preprocess(datas):
  clean = [change_user(text) for text in datas]
  clean = [remove_entity(text) for text in clean]
  clean = [remove_url(text) for text in clean]
  clean = [remove_noise_symbols(text) for text in clean]
  clean = [remove_stopwords(text) for text in clean]
  return clean

clean_tweet = preprocess(tweet)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [19]:
clean_tweet

['user woman shouldnt complain cleaning house . man always take trash .',
 'user boy dats cold.tyga dwn bad cuffin dat hoe 1st place',
 'user Dawg user ever fuck bitch start cry ? confused shit',
 'user user look like tranny',
 'user shit hear might true might faker bitch told ya',
 'user shit blows meclaim faithful somebody still fucking hoes',
 'user sit HATE another bitch got much shit going',
 'user cause Im tired big bitches coming us skinny girls',
 'might get ya bitch back thats',
 'user : hobbies include : fighting Mariam bitch',
 'Keeks bitch curves everyone lol walked conversation like . Smh',
 'Murda Gang bitch Gang Land',
 'hoes smoke losers ? yea . go IG',
 'bad bitches thing like',
 'bitch get',
 'bitch nigga miss',
 'bitch plz whatever',
 'bitch love',
 'bitches get cut everyday B',
 'black bottle bad bitch',
 'broke bitch cant tell nothing',
 'cancel bitch like Nino',
 'cant see hoes wont change',
 'fuck bitch dont even suck dick Kermit videos bout fuck IG',
 'got ya bi

In [20]:
X_train, X_test, y_train, y_test = train_test_split(clean_tweet,labels,test_size=0.2, random_state = 42)

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [30]:
X_train

[[1, 1, 125, 406, 56, 478, 12, 154, 541, 8360, 687, 1470, 215, 531, 130],
 [218, 159, 8361, 50, 50, 776, 522, 3690, 116, 225],
 [1, 1, 1, 16, 4491, 225, 2, 1],
 [1, 1, 1, 1, 1, 2, 11, 8362],
 [583, 444, 53, 2],
 [1, 1188, 48, 155, 40, 36, 32, 4, 324, 340, 6],
 [202, 1696, 2],
 [1, 1396, 570, 366, 114, 23, 584],
 [1, 1, 2, 95, 1316, 59],
 [633, 148, 633, 60, 445, 4, 269, 51, 455, 1189, 11, 8],
 [1, 181, 2749, 367, 289, 996, 2, 760],
 [1, 6, 11, 4492],
 [188, 196, 2, 13, 14],
 [353, 149, 50, 52, 242, 38, 90, 818, 649],
 [1, 1397, 532, 142],
 [593, 105, 2],
 [1, 533, 70, 8363, 2, 93, 121, 8364, 80, 50, 9, 915],
 [1, 1, 1, 2, 118, 101],
 [1, 59, 5, 278],
 [1, 2, 26, 3691, 150, 1],
 [8365, 397, 3692, 1398, 55, 3161, 2195, 141, 185],
 [38, 850, 32, 2, 916, 611, 117, 42, 502, 523],
 [851, 4493, 1, 1, 5779, 2442, 2750, 1399, 3],
 [1, 8366, 3693, 260, 371, 88, 4, 72, 2],
 [777, 650, 215, 2, 140],
 [1, 53, 8, 44, 64, 219, 208, 376, 1260, 64, 20, 189, 1846, 5780],
 [50, 235, 2196, 1697, 4494, 63,

In [31]:
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(seq) for seq in X_train)


In [32]:
max_length

26

In [33]:
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

y_train = to_categorical(y_train,num_classes=3)
y_test = to_categorical(y_test,num_classes=3)

In [34]:
X_train

array([[    0,     0,     0, ...,   215,   531,   130],
       [    0,     0,     0, ...,  3690,   116,   225],
       [    0,     0,     0, ...,   225,     2,     1],
       ...,
       [    0,     0,     0, ...,   384,  1814, 18655],
       [    0,     0,     0, ...,  4475,  1417,   375],
       [    0,     0,     0, ...,    17,   240,   685]], dtype=int32)

In [35]:
y_train

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [39]:
#Using Keras Tuner to find the best hyper - paramters for LSTM model
def build_model(hp):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size,
                        output_dim=hp.Choice('embedding_dim', [64, 128, 200]),
                        input_length=max_length))
  model.add(LSTM(units=hp.Int('lstm_units',32,128,step=32),
                 dropout=0.2, recurrent_dropout=0.2))
  model.add(Dropout(hp.Float('dropout_rate',0.3,0.6,step=0.1)))
  model.add(Dense(units=hp.Int('dense_units',64,128,step=32),
                  activation='relu'))
  model.add(Dense(3,activation='softmax'))
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  return model


In [40]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='hate_speech_fine_tuning'
)

In [41]:
tuner.search(X_train,
             y_train,
             epochs=5,
             validation_data=(X_test,y_test),
             batch_size=64)

Trial 5 Complete [00h 02m 47s]
val_accuracy: 0.8910631537437439

Best val_accuracy So Far: 0.8959047794342041
Total elapsed time: 00h 15m 38s


In [42]:
best_model  = tuner.get_best_models(num_models=1)[0]

In [43]:
loss,acc = best_model.evaluate(X_test,y_test)

155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8962 - loss: 0.3004


In [44]:
print(f"Best model accuracy on test set:{acc:4f}")

Best model accuracy on test set:0.895905


In [45]:
best_history  = best_model.history.history if hasattr(best_model,"history") else {}

In [46]:
if best_history:
  plt.plot(best_history['loss'],label='Loss')
  plt.plot(best_history['val_loss'],label='Validation Loss')
  plt.legend()
  plt.title("Best Model Loss")
  plt.show()

In [47]:
best_model.save("best_hate_speech_model.h5")


In [48]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [49]:
import json

metadata = {
    "max_length": max_length,
    "class_names": {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
}

with open("metadata.json", "w") as f:
    json.dump(metadata, f)


# **With Lemmatization**

> Just wanted to test the above code withe lemmatization using NLTK



In [51]:
# 1. Imports
import numpy as np
import pandas as pd
import os
import re
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical
import keras_tuner as kt

#Download NLTK Resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load Dataset
data = pd.read_csv("train.csv")
tweet = list(data['tweet'])
labels = list(data['class'])

# Preprocessing with Lemmatization + Stopwords
stop_words = set(stopwords.words('english'))
stop_words.add("rt")
lemmatizer = WordNetLemmatizer()

def remove_entity(text):
    return re.sub(r"&[^\s;]+;", "", text)

def change_user(text):
    return re.sub(r"@([^ ]+)", "user", text)

def remove_url(text):
    url_regex = r"http\S+|www\S+|https\S+"
    return re.sub(url_regex, '', text)

def remove_noise_symbols(text):
    return text.replace('"','').replace("'",'').replace('!','').replace('`','').replace('..','')

def preprocess_text(text):
    text = change_user(text)
    text = remove_entity(text)
    text = remove_url(text)
    text = remove_noise_symbols(text)
    tokens = word_tokenize(text)
    filtered = [lemmatizer.lemmatize(w.lower()) for w in tokens if w.lower() not in stop_words and w.isalpha()]
    return " ".join(filtered)

clean_tweet = [preprocess_text(t) for t in tweet]

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(clean_tweet, labels, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(seq) for seq in X_train)

X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

# Keras Tuner Model Builder
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=hp.Choice('embedding_dim', [64, 128, 200]),
                        input_length=max_length))
    model.add(LSTM(units=hp.Int('lstm_units', 32, 128, step=32),
                   dropout=0.2, recurrent_dropout=0.2))
    model.add(Dropout(hp.Float('dropout_rate', 0.3, 0.6, step=0.1)))
    model.add(Dense(units=hp.Int('dense_units', 64, 128, step=32), activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Run Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='hate_speech_lstm_tuning'
)

tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test),
             batch_size=64)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Best model accuracy on test set: {acc:.4f}")

# Save model and tokenizer
import pickle
best_model.save("best_lstm_model.h5")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("max_length.txt", "w") as f:
    f.write(str(max_length))

# Predict Function
def predict_tweet(tweet_text):
    tweet_text = preprocess_text(tweet_text)
    seq = tokenizer.texts_to_sequences([tweet_text])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = best_model.predict(padded)[0]
    predicted_class = np.argmax(pred)
    confidence = np.max(pred)
    class_names = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    print(f"Tweet: {tweet_text}")
    print(f"Predicted class: {class_names[predicted_class]} ({confidence:.2f} confidence)")

# Try it
predict_tweet("I hate people who do this!")
predict_tweet("Hope you are having a great day!")


Trial 5 Complete [00h 03m 19s]
val_accuracy: 0.89005446434021

Best val_accuracy So Far: 0.8934839367866516
Total elapsed time: 00h 16m 30s
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8927 - loss: 0.3103


Best model accuracy on test set: 0.8935
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Tweet: hate people
Predicted class: Offensive Language (0.43 confidence)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Tweet: hope great day
Predicted class: Neither (0.81 confidence)


In [52]:
clean_tweet

['user woman shouldnt complain cleaning house man always take trash',
 'user boy dat dwn bad cuffin dat hoe place',
 'user dawg user ever fuck bitch start cry confused shit',
 'user user look like tranny',
 'user shit hear might true might faker bitch told ya',
 'user shit blow meclaim faithful somebody still fucking hoe',
 'user sit hate another bitch got much shit going',
 'user cause im tired big bitch coming u skinny girl',
 'might get ya bitch back thats',
 'user hobby include fighting mariam bitch',
 'keeks bitch curve everyone lol walked conversation like smh',
 'murda gang bitch gang land',
 'hoe smoke loser yea go ig',
 'bad bitch thing like',
 'bitch get',
 'bitch nigga miss',
 'bitch plz whatever',
 'bitch love',
 'bitch get cut everyday b',
 'black bottle bad bitch',
 'broke bitch cant tell nothing',
 'cancel bitch like nino',
 'cant see hoe wont change',
 'fuck bitch dont even suck dick kermit video bout fuck ig',
 'got ya bitch tip toeing hardwood floor',
 'pussy lip like